In [1]:
import numpy as np
from matplotlib import pyplot as plt
from my_keras import *
from keras.datasets import cifar10

In [3]:
names_dict = {0: 'airplane', 1: 'automobile', 2: 'bird', 
              3: 'cat', 4: 'deer', 5: 'dog', 
              6: 'frog', 7: 'horse', 8: 'ship', 
              9: 'truck'}
              
(Xtrain, ytrain), (Xtest, ytest) = cifar10.load_data()


# summarize loaded dataset
print('Train: X=%s, y=%s' % (Xtrain.shape, ytrain.shape))
print('Test: X=%s, y=%s' % (Xtest.shape, ytest.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [4]:
def process(X):
    X_norm = X.reshape(len(X), X[0].size).astype(float)
    X_norm -= np.mean(X)
    X_norm /= np.std(X).astype(float)
    X_norm = np.hstack((np.ones((X_norm.shape[0], 1)), X_norm))
    return X_norm

In [5]:
Xtrain_n = process(Xtrain)
Xtest_n = process(Xtest)

testdata = [Xtest_n, ytest]

print(Xtrain_n.shape)

(50000, 3073)


In [6]:
reg1 = regularizers.L2(lambda_=.01)
reg2 = regularizers.L2(lambda_=.01)
reg3 = regularizers.L2(lambda_=.01)

model = models.Network()

model.add(layers.Input(Xtrain_n.shape[1], reg=reg1), scale=.01)
model.add(layers.Dense(100, activation=activations.sigmoid(), reg=reg2), scale=.01)
model.add(layers.Dense(100, activation=activations.sigmoid(),reg=reg3), scale=.01)
model.add(layers.LastLayer(10, activation=activations.linear()))
model.printLayers()

Layer 0: <class 'my_keras.layers.Input'>, neurons: 3073
Layer 1: <class 'my_keras.layers.Dense'>, neurons: 100
Layer 2: <class 'my_keras.layers.Dense'>, neurons: 100
Layer 3: <class 'my_keras.layers.LastLayer'>, neurons: 10


In [8]:
model.fit(Xtrain_n, ytrain, loss=losses.MSE_XOR(), opt=optimizers.SGD(alpha=.05), testdata=testdata,
          epochs=10, batch_size=100)

KeyboardInterrupt: 